In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import pandas as pd
import scipy.io as sio
import os
import shutil 
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [4]:
infile = "all_indtype.csv"
data = pd.read_csv(infile)
data

,indtype
0,1515
1,1711
2,4065
3,3538
4,1939
...,...
140537,1321
140538,3131
140539,4063
140540,1499


In [18]:
data_to_count = data['indtype']
intdic = {}
for i in data_to_count: #count appe time
    cur_key = int(i/100)
    if cur_key in intdic:
        if i in intdic[cur_key]:
            intdic[cur_key][i] += 1
        else:
            intdic[cur_key][i] = 1
    else:
        intdic[cur_key] = {i:1}
print(intdic)

{15: {1515: 75, 1512: 923, 1523: 213, 1513: 459, 1521: 159, 1590: 92, 1522: 182, 1550: 269, 1511: 122, 1529: 91, 1524: 56, 1514: 74, 1516: 39}, 17: {1711: 674, 1723: 911, 1790: 542, 1741: 75, 1773: 880, 1782: 770, 1744: 67, 1742: 515, 1781: 910, 1724: 644, 1775: 106, 1722: 1257, 1743: 190, 1721: 1431, 1712: 22, 1719: 204, 1774: 175, 1745: 36, 1789: 235, 1725: 157, 1726: 55, 1749: 118, 1729: 148, 1779: 45, 1783: 188, 1771: 439, 1763: 54, 1772: 101, 1762: 50, 1769: 30, 1713: 22, 1714: 34, 1761: 60}, 40: {4065: 227, 4024: 754, 4021: 491, 4043: 160, 4041: 1784, 4073: 505, 4061: 89, 4080: 36, 4099: 118, 4027: 366, 4074: 108, 4063: 83, 4069: 532, 4091: 79, 4022: 36, 4029: 460, 4092: 38, 4013: 272, 4023: 108, 4071: 293, 4049: 238, 4066: 33, 4012: 390, 4045: 240, 4072: 76, 4079: 93, 4011: 131, 4046: 158, 4062: 30, 4064: 134}, 35: {3538: 204, 3542: 567, 3569: 339, 3529: 304, 3519: 49, 3572: 276, 3571: 1508, 3564: 176, 3512: 144, 3521: 339, 3541: 612, 3531: 505, 3565: 422, 3526: 98, 3562: 111, 3

In [21]:
del_list = []
count_dic = {}
clusternum = 2000
for i in intdic:
    summ = 0
    for j in intdic[i]:
        summ += intdic[i][j]
    if summ < clusternumber:
        del_list.append(i)
    else:
        count_dic[i] = summ

print(count_dic)
print(del_list)
##drop small group
for i in del_list:
    del intdic[i]


{15: 2754, 17: 11145, 40: 8062, 35: 9211, 19: 3304, 26: 10546, 32: 2875, 36: 5704, 13: 8610, 20: 2524, 37: 6202, 31: 13362, 22: 4600, 18: 7708, 34: 8585, 30: 6424, 43: 3945, 14: 3832, 23: 3044, 41: 4367, 27: 3109}
[33, 29, 28, 24, 21, 25, 16, 42]


In [49]:
data = pd.read_csv("2001_big.csv")
order_dickey = list(intdic.keys())
order_dickey.sort()
#for i in order_dickey:
i = order_dickey[0]
avail_keygroup = list(intdic[i].keys())
avail_keygroup.sort()
data_to_processed = data[data['indtype'] == avail_keygroup[0]]
for j in avail_keygroup[1:]:
    df2 = data[data['indtype'] == j]
    data_to_processed = data_to_processed.append(df2)



In [50]:
data_to_processed

,crc,indtype,giovsaled,voexpg,fa,cwp,perseng,year,rnd,rndinten,expinten
3710,104005746,1311,24300,0,210,270,40,2001,0,0.000000,0.000000
3714,10400812X,1311,3090,0,4361,183,21,2001,0,0.000000,0.000000
5001,105220528,1311,51247,0,1740,301,53,2001,0,0.000000,0.000000
5004,105220624,1311,2048,0,710,11,48,2001,0,0.000000,0.000000
5009,10522131X,1311,40300,0,442,320,35,2001,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
139656,X21227183,1390,9050,0,560,410,58,2001,0,0.000000,0.000000
139929,YA3676739,1390,8000,0,580,106,11,2001,0,0.000000,0.000000
140297,hb282009x,1390,39200,39200,6576,408,240,2001,3,0.000077,1.000000
140505,x12947222,1390,8847,0,2029,210,120,2001,0,0.000000,0.000000


In [59]:
data_feature = [] ##[mean_giovsaled, mean_perseng,]
data_to_processed['ave_cwp'] = data_to_processed['cwp']/data_to_processed['perseng']
data_feature.append([data_to_processed['giovsaled'].mean(), data_to_processed['perseng'].mean(), data_to_processed['voexpg'].mean(),data_to_processed['expinten'].mean(),data_to_processed['fa'].mean(), data_to_processed['ave_cwp'].mean())

SyntaxError: invalid syntax (<ipython-input-59-1ea94ab49b7e>, line 3)

In [56]:
data_feature

[[42984.1831591173,
  174.89430894308944,
  4991.798606271777,
  0.10075010401372722,
  13588.771196283391]]

In [37]:
df1

,crc,indtype,giovsaled,voexpg,fa,cwp,perseng,year,rnd,rndinten,expinten
3710,104005746,1311,24300,0,210,270,40,2001,0,0.0,0.0
3714,10400812X,1311,3090,0,4361,183,21,2001,0,0.0,0.0
5001,105220528,1311,51247,0,1740,301,53,2001,0,0.0,0.0
5004,105220624,1311,2048,0,710,11,48,2001,0,0.0,0.0
5009,10522131X,1311,40300,0,442,320,35,2001,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
139761,X29454305,1311,11376,0,2728,136,262,2001,0,0.0,0.0
140141,gx0204421,1311,12414,0,14830,710,66,2001,0,0.0,0.0
140396,hu3932623,1311,30100,0,7200,220,35,2001,0,0.0,0.0
140402,k26214723,1311,15322,0,1347,511,148,2001,0,0.0,0.0


In [38]:
df2

,crc,indtype,giovsaled,voexpg,fa,cwp,perseng,year,rnd,rndinten,expinten
299,101130410,1312,116447,0,105294,4886,345,2001,2,0.000017,0.0
356,101164960,1312,10655,0,7052,1285,71,2001,0,0.000000,0.0
525,101507273,1312,63462,0,48598,3250,270,2001,0,0.000000,0.0
534,101516655,1312,45991,0,25948,3561,193,2001,0,0.000000,0.0
948,101897750,1312,71099,0,47187,3411,219,2001,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
140532,yg4339744,1312,11580,0,1280,190,38,2001,0,0.000000,0.0
140533,yg4339808,1312,10480,0,920,135,30,2001,0,0.000000,0.0
140534,yg4339816,1312,9783,0,1020,125,28,2001,0,0.000000,0.0
140535,yg4341254,1312,29018,0,2014,196,49,2001,0,0.000000,0.0


In [39]:
df2.append(df1)

,crc,indtype,giovsaled,voexpg,fa,cwp,perseng,year,rnd,rndinten,expinten
299,101130410,1312,116447,0,105294,4886,345,2001,2,0.000017,0.0
356,101164960,1312,10655,0,7052,1285,71,2001,0,0.000000,0.0
525,101507273,1312,63462,0,48598,3250,270,2001,0,0.000000,0.0
534,101516655,1312,45991,0,25948,3561,193,2001,0,0.000000,0.0
948,101897750,1312,71099,0,47187,3411,219,2001,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
139761,X29454305,1311,11376,0,2728,136,262,2001,0,0.000000,0.0
140141,gx0204421,1311,12414,0,14830,710,66,2001,0,0.000000,0.0
140396,hu3932623,1311,30100,0,7200,220,35,2001,0,0.000000,0.0
140402,k26214723,1311,15322,0,1347,511,148,2001,0,0.000000,0.0


In [ ]:
li = [1,2,3,1,2,4,5,6,3]
data = pd.DataFrame({'type':li, 'oo':li})
#print(data)
df2 =  pd.DataFrame([[5, 6], [7, 8]],columns = ['type', 'oo'])
data.append(df2,ignore_index=True)

print(data)

In [ ]:
with PdfPages('multipage_pdf.pdf') as pdf:
    #plt.figure(1)
    plt.plot(range(7), [3, 1, 4, 1, 5, 9, 2], 'r-o')
    plt.title('id = 1')
    pdf.savefig()  # saves the current figure into a pdf page
    plt.close()

    #plt.figure(2)
    plt.plot(range(7), [4, 2, 3, 4, 5, 6, 1], 'r-o')
    plt.title('id = 2')
    pdf.savefig()  # saves the current figure into a pdf page
    plt.close()
